In [1]:
import pandas as pd
import numpy as np

In [30]:
# global variables
RaterNum = 3
SubjectNum = None

## Calculation Reference  
Link: https://en.wikipedia.org/wiki/Fleiss%27_kappa

In [31]:
df1 = pd.read_csv("Hindi-Persian-human-annotation-1 - Sheet1.csv")
df2 = pd.read_csv("Hindi-Persian-human-annotation-2 - Sheet1.csv")
df3 = pd.read_csv("Hindi-Persian-human-annotation-3 - Sheet1.csv")
assert(len(df1)==len(df2))
assert(len(df2)==len(df3))
SubjectNum = len(df1)

In [32]:
extracted_col_from2 = df2.rename(columns={"Is loan?": "Is loan?2"})["Is loan?2"]
extracted_col_from3 = df3.rename(columns={"Is loan?": "Is loan?3"})["Is loan?3"]
df_all_raters = df1.join(extracted_col_from2).join(extracted_col_from3).fillna(0)
df_all_raters = df_all_raters.replace(to_replace=r'[xX]', value=1, regex=True)
#df_all_raters.dtypes
df_all_raters

,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3
0,3,अंदर,1,0,0
1,9,अगर,1,0,0
2,28,अब्र,1,1,1
3,58,आफ़त,1,1,1
4,59,आफ़ताब,1,1,1
...,...,...,...,...,...
120,1214,हयात,1,0,1
121,1220,हलवा,1,0,0
122,1236,हिलाल,1,0,1
123,1243,हैरत,1,1,1


In [33]:
def LoanCount(row):
    return row['Is loan?'] + row['Is loan?2']  + row['Is loan?3']    
df_all_raters['loan_num'] = df_all_raters.apply(lambda row: LoanCount(row), axis=1)
df_all_raters
df_all_raters['not_loan_num'] = df_all_raters.apply(lambda row: RaterNum -row['loan_num'], axis=1)
df_all_raters
#df_all_raters.to_csv("temp_check.csv")

,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,loan_num,not_loan_num
0,3,अंदर,1,0,0,1,2
1,9,अगर,1,0,0,1,2
2,28,अब्र,1,1,1,3,0
3,58,आफ़त,1,1,1,3,0
4,59,आफ़ताब,1,1,1,3,0
...,...,...,...,...,...,...,...
120,1214,हयात,1,0,1,2,1
121,1220,हलवा,1,0,0,1,2
122,1236,हिलाल,1,0,1,2,1
123,1243,हैरत,1,1,1,3,0


In [35]:
def CalculatePi(row):
    numerator = row['loan_num']**2 + row['not_loan_num']**2 - RaterNum
    denominator = RaterNum*(RaterNum-1)
    return numerator/denominator
df_all_raters['Pi'] = df_all_raters.apply(lambda row: CalculatePi(row), axis=1)
df_all_raters

,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,loan_num,not_loan_num,Pi
0,3,अंदर,1,0,0,1,2,0.333333
1,9,अगर,1,0,0,1,2,0.333333
2,28,अब्र,1,1,1,3,0,1.000000
3,58,आफ़त,1,1,1,3,0,1.000000
4,59,आफ़ताब,1,1,1,3,0,1.000000
...,...,...,...,...,...,...,...,...
120,1214,हयात,1,0,1,2,1,0.333333
121,1220,हलवा,1,0,0,1,2,0.333333
122,1236,हिलाल,1,0,1,2,1,0.333333
123,1243,हैरत,1,1,1,3,0,1.000000


In [41]:
Pj_loan = df_all_raters['loan_num'].sum()/(SubjectNum*RaterNum)
Pj_not_loan = df_all_raters['not_loan_num'].sum()/(SubjectNum*RaterNum)
Pe_avg = Pj_loan**2 + Pj_not_loan**2
Pi_avg = df_all_raters['Pi'].sum()/SubjectNum

In [42]:
kappa = (Pi_avg-Pe_avg)/(1-Pe_avg)
kappa

0.047619047619047054

In [38]:
SubjectNum = len(df1)
SubjectNum

125

In [40]:
df_all_raters['not_loan_num'].sum()

105

# Record

Hindi_Persian Fleiss Kappa value = 0.047619047619047054 (Based on three raters)